In [1]:
# %pip install torch torchvision torchaudio
# %pip install bitsandbytes datasets accelerate transformers peft

### Helper Function for Comparing Results

In [2]:
from IPython.display import display, Markdown

def generate(model, tokenizer, context, question):
    batch = tokenizer(f"**CONTEXT:**\n{context}\n\n**QUESTION:**\n{question}\n\n**ANSWER:**\n", return_tensors='pt', return_token_type_ids=False)
    # batch = batch.to(device='cuda')

    #raw model
    return tokenizer.decode(model.generate(**batch, max_new_tokens=200)[0], skip_special_tokens=True)

def compare_inference(model, tokenizer, context, question):
    out = generate(model, tokenizer, context, question)
    display(Markdown("# Finetuned Model\n"))
    display(Markdown(out))
    model.disable_adapters()
    out = generate(model, tokenizer, context, question)
    display(Markdown("# Raw Model\n"))
    display(Markdown(out))

### Importing dependencies and downloading pre-trained model

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "bigscience/bloom-560m"
#peft_model_id = "./checkpoint/BLOOM-560m-LoRA"
model_id = "facebook/opt-350m"
peft_model_id = "./checkpoint/opt-350m-lora"

#loading model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # torch_dtype=torch.float32,
    device_map='auto',
)

#loading tokenizer for this model (which turns text into an input for the model)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/simon/anaconda3/envs/wheels/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/simon/anaconda3/envs/wheels/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/home/simon/anaconda3/envs/wheels/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 644/644 [00:00<00:00, 2.86MB/s]
pytorch_model.bin: 100%|██████████| 663M/663M [01:00<00:00, 11.0MB/s] 
tokenizer_config.json: 100%|██████████| 685/685 [00:00<00:00, 4.29MB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 1.27MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 10.7MB/s]
special_tokens_map.json: 100%|██████████| 441/441 [00:00<00:00, 2.83MB/s]


In [4]:
# model.load_adapter(peft_model_id)

# context = "you are a math wizard"
# question = "what is 1+1 equal to?"
# compare_inference(model, tokenizer, context, question)

### Setting up LoRA using parameter efficient fine tuning

In [5]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.1,
    # target_modules=["query_key_value"],
    bias="none",
    task_type="CAUSAL_LM"
)

model.add_adapter(peft_config)

### Comparing parameters before and after LoRA

In [6]:
trainable_params = 0
all_param = 0

#iterating over all parameters
for _, param in model.named_parameters():
    #adding parameters to total
    all_param += param.numel()
    #adding parameters to trainable if they require a graident
    if param.requires_grad:
        trainable_params += param.numel()

#printing results
print(f"trainable params: {trainable_params}")
print(f"all params: {all_param}")
print(f"trainable: {100 * trainable_params / all_param:.2f}%")

trainable params: 786432
all params: 331982848
trainable: 0.24%


### Loading and reformating SQUAD dataset

In [7]:
from datasets import load_dataset
qa_dataset = load_dataset("squad_v2")

In [8]:
def create_prompt(context, question, answer):
  if len(answer["text"]) < 1:
    answer = "Cannot Find Answer"
  else:
    answer = answer["text"][0]
  prompt_template = f"CONTEXT:\n{context}\n\nQUESTION:\n{question}\n\nANSWER:\n{answer}</s>"
  return prompt_template

#applying the reformatting function to the entire dataset
mapped_qa_dataset = qa_dataset.map(lambda samples: tokenizer(create_prompt(samples['context'], samples['question'], samples['answers'])))

Map: 100%|██████████| 11873/11873 [00:05<00:00, 2138.38 examples/s]


### Fine tuning

This code is largly co-opted. In the absence of a rigid validation
procedure, the best practice is to just copy a successful tutorial or,
better yet, directly from the documentation.

In [9]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_qa_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=100,
        learning_rate=1e-3,        
        logging_steps=10,
        output_dir='results',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
# peft_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

model.save_pretrained(peft_model_id)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.180400
20,3.064600
30,3.010900
40,2.944500
50,2.878800
60,2.850200
70,2.894900
80,2.811600
90,2.863800
100,2.838700


/home/simon/anaconda3/envs/wheels/lib/python3.10/site-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


### Test

In [10]:
context = "You are a monster, and you eat yellow legos."
question = "What is the best food?"

compare_inference(model, tokenizer, context, question)

# Finetuned Model


**CONTEXT:**
You are a monster, and you eat yellow legos.

**QUESTION:**
What is the best food?

**ANSWER:**
Cannot Find Answer

# Raw Model


**CONTEXT:**
You are a monster, and you eat yellow legos.

**QUESTION:**
What is the best food?

**ANSWER:**
I like to eat a lot of yellow legos.

**QUESTION:**
What is the best way to get a good night's sleep?

**ANSWER:**
I like to sleep in a bed with a lot of yellow legos.

**QUESTION:**
What is the best way to get a good night's sleep?

**ANSWER:**
I like to sleep in a bed with a lot of yellow legos.

**QUESTION:**
What is the best way to get a good night's sleep?

**ANSWER:**
I like to sleep in a bed with a lot of yellow legos.

**QUESTION:**
What is the best way to get a good night's sleep?

**ANSWER:**
I like to sleep in a bed with a lot of yellow legos.

**QUESTION:**
What is the best way

In [11]:
context = "you are a math wizard"
question = "what is 1+1 equal to?"

compare_inference(model, tokenizer, context, question)

# Finetuned Model


**CONTEXT:**
you are a math wizard

**QUESTION:**
what is 1+1 equal to?

**ANSWER:**
1+1 is equal to 1+1.

**CONTEXT:**
1+1 is equal to 1+1.

**ANSWER:**
1+1 is equal to 1+1.

**ANSWER:**
1+1 is equal to 1+1.

**ANSWER:**
1+1 is equal to 1+1.

**ANSWER:**
1+1 is equal to 1+1.

**ANSWER:**
1+1 is equal to 1+1.

**ANSWER:**
1+1 is equal to 1+1.

**ANSWER:**
1+1 is equal to 1+1.

**ANSWER:**
1+1 is equal to 1+1.

**ANSWER:**
1+1 is equal to 1+1.

**ANSWER:**
1+

# Raw Model


**CONTEXT:**
you are a math wizard

**QUESTION:**
what is 1+1 equal to?

**ANSWER:**
1+1=1

**CONTEXT:**
1+1=1

**QUESTION:**
what is 1+1 equal to?

**ANSWER:**
1+1=1

**CONTEXT:**
1+1=1

**QUESTION:**
what is 1+1 equal to?

**ANSWER:**
1+1=1

**QUESTION:**
what is 1+1 equal to?

**ANSWER:**
1+1=1

**QUESTION:**
what is 1+1 equal to?

**ANSWER:**
1+1=1

**QUESTION:**
what is 1+1 equal to?

**ANSWER:**
1+1=1

**QUESTION:**
what is 1+1 equal to?

**ANSWER:**


In [12]:
context = "Answer the riddle"
question = "What gets bigger the more you take away?"

compare_inference(model, tokenizer, context, question)

# Finetuned Model


**CONTEXT:**
Answer the riddle

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**CONTEXT:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**ANSWER:**

**QUESTION:**


# Raw Model


**CONTEXT:**
Answer the riddle

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**QUESTION:**
What gets bigger the more you take away?

**ANSWER:**

**QUESTION:**
What